In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import string
import re
import collections
from sklearn import  preprocessing
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, mean_absolute_error, confusion_matrix
import optuna

%matplotlib inline
import itertools





import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
sns.set_style("whitegrid")
import warnings




In [5]:
train = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/train.json.zip')
test = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/test.json.zip')

# Basic Insight of Dataset (Базовое представление о наборе данных)


In [6]:
train. head(3)

In [11]:
train.shape

In [12]:
test.shape


In [13]:
print(train.columns.values)

In [14]:
print(test.columns.values)

In [15]:
train.describe()

In [16]:
train.describe(include = "all")

In [17]:
train.info()

In [18]:
test.info()

In [19]:
train.isnull().sum()

In [20]:
test.isnull().sum()

# Data visualization and pre-processing (Визуализация данных и предварительная обработка)

### 'interest_level'


In [21]:
train.groupby(train['interest_level']).mean()

In [22]:
train['interest_level'].value_counts()

In [7]:
sns.set_style('whitegrid')
sns.countplot(x='interest_level',data=train,palette='RdBu_r')

### 'bathrooms'

In [8]:
# Value Counts
train['bathrooms'].value_counts()



In [9]:
train.groupby(['bathrooms']) ['interest_level'].value_counts(normalize=True)

In [10]:
train.pivot_table('price', 'bathrooms', 'interest_level', 'count').plot(kind='bar', stacked=True)

### 'bedrooms'

In [27]:
# Value Counts
train['bedrooms'].value_counts()

In [28]:
train.groupby(['bedrooms']) ['interest_level'].value_counts(normalize=True)

In [29]:
train.pivot_table('price', 'bedrooms', 'interest_level', 'count').plot(kind='bar', stacked=True)

In [30]:
# 'bathrooms' and 'bedrooms'
fig, axes = plt.subplots(ncols=2)
train.pivot_table('price', ['bathrooms'], 'interest_level', 'count').plot(ax=axes[0], title='bathrooms')
train.pivot_table('price', ['bedrooms'], 'interest_level', 'count').plot(ax=axes[1], title='bedrooms')

### 'building_id'

In [31]:
# Most advertised buildings
train.building_id.value_counts().nlargest(10)

### 'created'

In [11]:
# Convertion to Python Date  (Преобразование в дату Python)
train.created = pd.to_datetime(train.created, format='%Y-%m-%d %H:%M:%S')
test.created = pd.to_datetime(test.created, format='%Y-%m-%d %H:%M:%S')

In [13]:
#  Month, Day of Week and Hour Features  (Месяц, день недели и часы)
train['month'] = train.created.dt.month
train['day_of_week'] = train.created.dt.weekday
train['hour'] = train.created.dt.hour

train.head(3)

In [49]:
#  Month, Day of Week and Hour Features  (Месяц, день недели и часы)
test['month'] = test.created.dt.month
test['day_of_week'] = test.created.dt.weekday
test['hour'] = test.created.dt.hour

test.head(2)

In [14]:
# Iterest per month
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="month", hue="interest_level", hue_order=['low', 'medium', 'high'],
                   data=train);
plt.xlabel('Month');
plt.ylabel('Number of occurrences')

# Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")


In [36]:
# Iterest per Day of Week
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="day_of_week", hue="interest_level",
                   hue_order=['low', 'medium', 'high'], data=train);
plt.xlabel('Day of Week');
plt.ylabel('Number of occurrences');

### Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")

In [37]:
# Iterest per Hour
fig = plt.figure(figsize=(12,6))
sns.countplot(x="hour", hue="interest_level", hue_order=['low', 'medium', 'high'], data=train);
plt.xlabel('Hour');
plt.ylabel('Number of occurrences');

### 'display_address'

In [15]:
# Number of unique Display Addresses
print('Number of Unique Display Addresses is {}'.format(train.display_address.value_counts().shape[0]))

In [16]:
# 20 most popular Display Addresses
train.display_address.value_counts().nlargest(20)

In [17]:
# Top 20 northernmost points (Топ-20 самых северных точек)
train.latitude.nlargest(20)

In [41]:
# Top 20 westernmost points  (Топ-20 самых западных точек)
train.latitude.nsmallest(20)

In [42]:
# Top 20 easternmost points  (самых восточных точек)
train.longitude.nlargest(20)

In [18]:
# Top 20 westernmost points  (Топ-20 самых западных точек)
train.longitude.nsmallest(20)

In [19]:
# Rent interest graph of New-York
sns.lmplot(x="longitude", y="latitude", fit_reg=False, hue='interest_level',
           hue_order=['low', 'medium', 'high'], size=9, scatter_kws={'alpha':0.4,'s':30},
           data=train[(train.longitude>train.longitude.quantile(0.1))
                        &(train.longitude<train.longitude.quantile(0.9))
                        &(train.latitude>train.latitude.quantile(0.1))                           
                        &(train.latitude<train.latitude.quantile(0.9))]);
plt.xlabel('Longitude');
plt.ylabel('Latitude');

### 'manager_id'

In [20]:
train.manager_id.value_counts().nlargest(10)

In [21]:
# Let's get a list of top 10 managers
top10managers = train.manager_id.value_counts().nlargest(10).index.tolist()
# ...and plot number of different Interest Level rental adverts for each of them
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="manager_id", hue="interest_level",
                   data=train[train.manager_id.isin(top10managers)]);
plt.xlabel('Manager');
plt.ylabel('Number of advert occurrences');
### Manager_ids are too long. Let's remove them
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off');


plt.xticks([])
# Adding percents over bars
height = [0 if np.isnan(p.get_height()) else p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 20,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")

### 'photos_number'

In [22]:
# count of photos 
train["photos_number"] = train["photos"].apply(len)
train["photos_number"].value_counts()

In [52]:
# count of photos 
test["photos_number"] = test["photos"].apply(len)
test.head(2)

### 

### price

In [24]:
# Value Counts
train['price'].value_counts()

In [25]:
# Price exploration
prices=train.groupby('interest_level', as_index=False)['price'].mean()
colors = ['lightcoral','gold','lightblue']

fig=plt.figure(figsize=(8,6))
plt.bar(prices.interest_level, prices.price, color=colors, width=0.5, alpha=0.8)
#set titles
plt.xlabel('Interest level')
plt.ylabel('Average price')
plt.title('Average price across interest level')
plt.show()

In [50]:
# Price exploration
fig = plt.figure(figsize=(12,12))
# Price distribution
sns.distplot(train.price[train.price<=train.price.quantile(0.99)], ax=plt.subplot(211));
plt.xlabel('Price');
plt.ylabel('Density');

### Average Price per Interest Level
sns.barplot(x="interest_level", y="price", order=['low', 'medium', 'high'],
            data=train, ax=plt.subplot(223));
plt.xlabel('Interest Level');
plt.ylabel('Price');

### Violinplot of price for every Interest Level
sns.violinplot(x="interest_level", y="price", order=['low', 'medium', 'high'],
               data=train[train.price<=train.price.quantile(0.99)],
               ax=plt.subplot(224));
plt.xlabel('Interest Level');
plt.ylabel('Price');

### 'description'

In [26]:
train['description'].iloc[0]

In [31]:
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION  (УДАЛИТЬ НЕНУЖНЫЕ СЛОВА ИЗ ОПИСАНИЯ)

train['description'] = train['description'].apply(lambda x: x.replace("<br />", ""))
train['description'] = train['description'].apply(lambda x: x.replace("br", ""))
train['description'] = train['description'].apply(lambda x: x.replace("<p><a", ""))
train['description'].iloc[0]

In [30]:
test['description'].iloc[0]

In [32]:
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION  (УДАЛИТЬ НЕНУЖНЫЕ СЛОВА ИЗ ОПИСАНИЯ)

test['description'] = test['description'].apply(lambda x: x.replace("<br />", ""))
test['description'] = test['description'].apply(lambda x: x.replace("br", ""))
test['description'] = test['description'].apply(lambda x: x.replace("<p><a", ""))
test['description'].iloc[0]

In [33]:
# description contains email
regex = r'[\w\.-]+@[\w\.-]+'
train['has_email'] = train['description'].apply(lambda x: 1 if re.findall(regex, x) else 0)
train['has_email'].value_counts()

In [34]:
# description contains email
regex = r'[\w\.-]+@[\w\.-]+'
test['has_email'] = test['description'].apply(lambda x: 1 if re.findall(regex, x) else 0)
test['has_email'].value_counts()

In [35]:
# description contains phone
train['has_phone'] = train['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)
train['has_phone'].value_counts()

In [58]:
# description contains phone
test['has_phone'] = test['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)
test['has_phone'].value_counts()

### 'features'

In [36]:
# count of "features" 
train["num_features"] = train["features"].apply(len)
train["num_features"].value_counts()

In [37]:
# count of "features" 
test["num_features"] = test["features"].apply(len)
test["num_features"].value_counts()

In [38]:
# CONVERT LOWER ALL OF WORDS
train[["features"]] = train[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])
train[["features"]]


In [39]:
# CONVERT LOWER ALL OF WORDS
test[["features"]] = test[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])
test[["features"]]


## MOST FREQUENT FEATURES EXTRACTION  (ИЗВЛЕЧЕНИЕ НАИБОЛЕЕ ЧАСТОТЫХ ФУНКЦИЙ)

In [40]:
feature_value_train = train['features'].tolist()
feature_value_test = test['features'].tolist()


In [41]:
feature_lst_train = []
for i in range(len(feature_value_train)):
    feature_lst_train += feature_value_train[i]
    
uniq_feature_train = list(set(feature_lst_train))

# print(uniq_feature) #all unique features
len(uniq_feature_train)
    
    

In [42]:
feature_lst_test = []
for i in range(len(feature_value_test)):
    feature_lst_test += feature_value_test[i]
    
uniq_feature_test = list(set(feature_lst_test))

# print(uniq_feature) #all unique features
len(uniq_feature_test)

In [43]:
# see the frequency of each feature
import collections
def most_common(lst):
    features = collections.Counter(lst)
    feature_value = features.keys()
    frequency = features.values()
    data = [('feature_value', feature_value),
            ('frequency', frequency),]    
    df = pd.DataFrame.from_dict(dict(data))
    return df.sort_values(by = 'frequency', ascending = False)

df_features_train = most_common(feature_lst_train)

df_features_test = most_common(feature_lst_test)

df_features_train


In [44]:
df_features_train.head(20)

In [45]:
df_features_test

In [46]:
def newColumn(name, df, series):
    feature = pd.Series(0,df.index,name = name)# data : 0
    for row,word in enumerate(series):
        if name in word:
            feature.iloc[row] = 1
    df[name] = feature # feature : series ; value in series : 1 or 0
    return df

# select features based on frequency
facilities = [
'elevator', 'cats allowed', 'hardwood floors', 'dogs allowed', 'doorman', 'dishwasher', 'no fee', 'laundry in building', 'fitness center',
'laundry in unit', 'pre-war', 'roof deck', 'outdoor space','dining room', 'high speed internet', 'balcony', 'swimming pool',
'new construction', 'terrace']
for name in facilities:
    train = newColumn(name, train, train['features'])
    test = newColumn(name, test, test['features'])
train.head(5)

In [53]:
test.head(3)

In [71]:
#WORDCLOUD FOR DESCRIPTION AND DISPLAY ADDRESS
#Preprocessing
text = ''
text_da = ''
text_desc = ''
text_str = ''
for ind, row in train.iterrows():
    for feature in row['features']:
        text = " ".join([text, "_".join(feature.strip().split(" "))])
    text_da = " ".join([text_da,"_".join(row['display_address'].strip().split(" "))])
    text_desc = " ".join([text_desc, row['description']])
    text_str = " ".join([text_str, row['street_address']])
text = text.strip()
text_da = text_da.strip()
text_desc = text_desc.strip()
text_str = text_str.strip()


# wordcloud for features
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for features", fontsize=30)
plt.axis("off")
plt.show()




# wordcloud for display address
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text_da)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for Display Address", fontsize=30)
plt.axis("off")
plt.show()


# wordcloud for description
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text_desc)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for Description", fontsize=30)
plt.axis("off")
plt.show()

# wordcloud for street address
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text_str)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for Street Address", fontsize=30)
plt.axis("off")
plt.show()

## Our target 'INTEREST LEVEL' is an object.( Наша цель «УРОВЕНЬ ИНТЕРЕСА» — это объект.)
## Let's convert to the numeric to analyze easily (Давайте перейдем к числовому, чтобы легко анализировать)
## 0 : low; 1 : medium; 2 : high


In [54]:
train['target'] = train['interest_level'].apply(lambda x: 0 if x=='low' 
                                                      else 1 if x=='medium' 
                                                      else 2) 
# train_df['low'] = train_df['interest_level'].apply(lambda x: 1 if x=='low' else 0)
# train_df['medium'] = train_df['interest_level'].apply(lambda x: 1 if x=='medium' else 0)
# train_df['high'] = train_df['interest_level'].apply(lambda x: 1 if x=='high' else 0)

## DROP UNNECESSARY COLUMNS  (УДАЛИТЬ НЕНУЖНЫЕ СТОЛБЦЫ)

In [56]:
# TRAINING DATASET
train.drop('interest_level', axis=1, inplace=True)
train.drop('created', axis=1, inplace=True)
train.drop('description', axis=1, inplace=True)
train.drop('features', axis=1, inplace=True)
train.drop('photos', axis=1, inplace=True)


# TEST DATASET
test.drop('created', axis=1, inplace=True)
test.drop('description', axis=1, inplace=True)
test.drop('features', axis=1, inplace=True)
test.drop('photos', axis=1, inplace=True)

# LABEL ECONDING FOR CATEGORICAL VARIABLES (КОДИРОВАНИЕ МЕТКИ ДЛЯ CATEGORICAL ПЕРЕМЕННЫХ)

In [59]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[f].values) + list(test[f].values))
            train[f] = lbl.transform(list(train[f].values))
            test[f] = lbl.transform(list(test[f].values))
            


# XGBOOST  (алгоритм градиентного бустинга на деревьях)

In [60]:

X = train.drop(['target'], axis = 1)
y = train.target


X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                   stratify = y)

# Импорт XGBoost и создание необходимых объектов.
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)






In [61]:
kf = KFold(n_splits=5, shuffle=False)

X_train = X_train.values
y_train = y_train.values
scores = []

for train, test in kf.split(X_train, y_train):
    model = XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth = 10)
    model.fit(X_train[train], y_train[train])
    scores.append(model.score(X_train[test], y_train[test]))

In [62]:
def objective(trial):
    params = {
        'booster':trial.suggest_categorical('booster', ['gbtree', 'dart', 'gblinear']),
        'learning_rate':trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        'max_depth':trial.suggest_int("max_depth", 3, 11),
        'subsample':trial.suggest_uniform("subsample", 0.0, 1.0),
        'colsample_bytree':trial.suggest_uniform("colsample_bytree", 0.0, 1.0),
    }

    model = XGBClassifier(**params)
    cv = KFold(n_splits=3, shuffle=True, random_state=None)
    scorer = make_scorer(f1_score, greater_is_better=True)
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    f1_scores = f1_score(y_test, pred_labels, average='micro')
    return f1_scores

In [63]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

In [64]:
new_params = study.best_params

new_model = XGBClassifier(**new_params)
new_model.fit(X, y)
preds = new_model.predict(X_test)

print('Optimized SuperLearner accuracy: ', accuracy_score(y_test, preds))
print('Optimized SuperLearner f1-score: ', f1_score(y_test, preds, average='micro'))


In [65]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [66]:
print("All of accuracies")
print(scores)

print("Mean of accuracies")
print(np.mean(scores))